In [5]:
!pip install langgraph langchain tavily-python langchain-community langchain-community

  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)


In [42]:
import os
import json
from typing import Dict, List, Any, Callable

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END
from tavily import TavilyClient


In [9]:
!pip3 install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 2.0 MB/s eta 0:00:00a 0:00:01
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.6/309.6 kB 2.7 MB/s eta 0:00:00a 0:00:01


In [11]:
from openai import OpenAI
client1 = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")  # html to json


In [12]:
model = r"/Users/karthiknamboori/.cache/lm-studio/models/lmstudio-community/llama-3.2-3b-instruct"


In [13]:
def get_completion(prompt, client=client1, model=model):
    """
    given the prompt, obtain the response from LLM hosted by LM Studio as a server
    :param prompt: prompt to be sent to LLM server
    :return: response from the LLM
    """
    prompt = [
        {"role": "user", "content": prompt}
    ]
    completion = client.chat.completions.create(
        model=model,
        messages=prompt,
        temperature=0.0,
        stream=True,
    )

    new_message = {"role": "assistant", "content": ""}

    for chunk in completion:
        if chunk.choices[0].delta.content:
            # print(chunk.choices[0].delta.content, end="", flush=True)
            val = chunk.choices[0].delta.content
            new_message["content"] += val

    # print(type()
    val = new_message["content"]  # .split("<end_of_turn>")[0]

    return val

In [52]:
prompt = """
    You are a political leader and your party is trying to win the general elections in India. 
    You are given an LLM that can provide you the analytics using the past historical data given to it.
    In particular the LLM has been provided data on which party won each constituency out of 545 and which assembly segment within the main constituency is more favorable.
    It also has details of votes polled by every candidate.
    Tell me 10 questions that you want to ask the LLM.
    """
results = get_completion(prompt)
print(results)

As a political leader, I'd like to utilize the capabilities of this LLM to gain valuable insights that can help my party win the general elections in India. Here are 10 questions I'd like to ask:

1. **Which constituencies have historically voted in favor of our party, and what are the key factors contributing to their success?**

This question would help me identify areas of strength and understand the underlying reasons for our party's past victories in these constituencies.

2. **Which assembly segments within each constituency are more favorable to our party, and what are the demographic characteristics of these areas?**

This question would enable me to focus on specific segments within each constituency and tailor my campaign strategies accordingly.

3. **What is the average vote share of our party in each constituency, and how does it compare to the national average?**

This question would provide me with a comprehensive understanding of our party's performance across different 

In [15]:
# State for tracking conversation and context
class GraphState(BaseModel):
    query: str = Field(default="")
    chat_history: List[Dict[str, str]] = Field(default_factory=list)
    context: List[str] = Field(default_factory=list)
    result: str = Field(default="")
    retry_count: int = Field(default=0)

In [16]:
# Load pharmaceutical data (you'll need to replace this with your actual JSON data)
def load_pharmaceutical_data(file_path):
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except Exception as e:
        print(f"Error loading pharmaceutical data: {e}")
        return {}

In [17]:
# RAG Node for searching pharmaceutical database
def pharmaceutical_rag(state: GraphState) -> Dict:
    print("Running Pharmaceutical RAG...")
    
    # Load your pharmaceutical database (replace with actual path)
    pharma_data = load_pharmaceutical_data('pharmaceutical_database.json')
    
    # Simple search through pharmaceutical data
    matching_entries = []
    for entry in pharma_data:
        if state.query.lower() in json.dumps(entry).lower():
            matching_entries.append(entry)
    
    if matching_entries:
        context = [json.dumps(entry) for entry in matching_entries]
        return {
            "context": context,
            "result": f"Found {len(matching_entries)} matching entries."
        }
    
    return {"context": [], "result": "No matching entries found in database."}


In [33]:
os.environ["TAVILY_API_KEY"] = "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 "

In [44]:
# Web Search Node using Tavily
def web_search(state: GraphState) -> Dict:
    print("Performing Web Search...")
    try:
        tavily_client = TavilyClient(api_key="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")
        response = tavily_client.search(state.query)
        
        context = [
            f"URL: {result['url']}\nContent: {result['content']}"
            for result in response['results']
        ]
        
        return {
            "context": context,
            "result": f"Found {len(context)} web search results."
        }
    except Exception as e:
        print(f"Web search error: {e}")
        return {"context": [], "result": "Web search failed."}



In [29]:
# test web search
def test_web_search(query):
    print("Performing Web Search...")
    try:
        tool = TavilySearchResults(max_results=3)
        search_results = tool.invoke("What's a 'node' in LangGraph?")
        
        context = [
            f"URL: {result['url']}\nContent: {result['content']}"
            for result in search_results
        ]
        
        return {
            "context": context,
            "result": f"Found {len(context)} web search results."
        }
    except Exception as e:
        print(f"Web search error: {e}")
        return {"context": [], "result": "Web search failed."}

In [41]:
# response = tavily_client.search("Who is Varun MS, NeurIPS?")

{'query': 'Who is Varun MS, NeurIPS?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': '\u202aVarun Nair\u202c - \u202aGoogle Scholar\u202c',
   'url': 'https://scholar.google.com/citations?user=R3SqGgMAAAAJ',
   'content': 'Varun Nair. Research at Harvey. Verified email at duke.edu. machine learning computer vision NLP. Articles Cited by Public access Co-authors. Title. Sort. ... NeurIPS, 2021. 3: 2021: CONSCENDI: A Contrastive and Scenario-Guided Distillation Approach to Guardrail Models for Virtual Assistants.',
   'score': 0.9893885,
   'raw_content': None},
  {'title': 'Varun Jampani - Home',
   'url': 'https://varunjampani.github.io/',
   'content': "Varun Jampani. Toggle navigation Varun Jampani. Home; Publications; ... May, 2024 : Area chairing for BMVC'24 and NeurIPS'24. March, 2024 : Papers on '3D awareness in foundation models', 'Multi-view diffusion', 'One-shot affordance learning', 'Neural BRDF decomposition',",
   'score': 0.9588471,
 

In [30]:
test_web_search("Who is varun MS ?")

Performing Web Search...
Web search error: string indices must be integers, not 'str'


{'context': [], 'result': 'Web search failed.'}

In [67]:
# LLM Node for generating final answer
def generate_answer(state: GraphState) -> Dict:
    print("Generating Answer...")
    
    # Combine context and create prompt
    context_str = "\n---\n".join(state.context)
    
    # Construct a comprehensive prompt
    full_prompt = f"""You are a helpful pharmaceutical information assistant. 
    Use the following context to answer the user's query precisely and accurately.

    Query: {state.query}

    Context:
    {context_str}

    Guidelines:
    - Provide a detailed and accurate response based on the context
    - If information is insufficient, clearly state more research is needed
    - Include relevant details about the pharmaceutical topic
    - Be clear, concise, and informative

    Your detailed response:"""
    
    try:
        # Use the get_completion function
        response = get_completion(full_prompt)
        
        return {
            "result": response,
            "chat_history": state.chat_history + [
                {"role": "user", "content": state.query},
                {"role": "ai", "content": response}
            ]
        }
    except Exception as e:
        print(f"Answer generation error: {e}")
        return {"result": "Could not generate an answer."}

# Router Node to decide next action
def route_query(state: GraphState) -> str:
    print("Routing Query...")
    
    # If retry count is high, end the process
    if state.retry_count >= 2:
        return END
    
    # Check if pharmaceutical RAG provided a good answer
    if state.context and "No matching entries" not in state.result:
        return "generate_answer"
    
    # Otherwise, try web search
    return "web_search"

In [68]:
# Build the Graph
def create_graph():
    workflow = StateGraph(GraphState)
    
    # Add nodes
    workflow.add_node("pharmaceutical_rag", pharmaceutical_rag)
    workflow.add_node("web_search", web_search)
    workflow.add_node("generate_answer", generate_answer)
    
    # Set entry point
    workflow.set_entry_point("pharmaceutical_rag")
    
    # Add conditional edges
    workflow.add_conditional_edges(
        "pharmaceutical_rag",
        route_query,
        {
            "web_search": "web_search",
            "generate_answer": "generate_answer",
            END: END
        }
    )
    
    workflow.add_conditional_edges(
        "web_search",
        route_query,
        {
            "web_search": "web_search",
            "generate_answer": "generate_answer",
            END: END
        }
    )
    
    # Compile the graph
    return workflow.compile()

In [69]:
# # Main execution
# def main():
#     # Create the graph
#     
    
#     # # Example queries
#     # queries = [
#     #     "What is the composition and primary use of Paracetamol?",
#     #     "Can I take Ibuprofen if I have a history of stomach ulcers?"
#     # ]
    
#     # for query in queries:
#     #     print(f"\n--- Processing Query: {query} ---")
#     #     result = app.invoke({"query": query})
#     #     print("\nFinal Result:", result['result'])

# if __name__ == "__main__":
#     main()
app = create_graph()
queries = [
"What is the composition and primary use of Paracetamol?",
"Can I take Ibuprofen if I have a history of stomach ulcers?"
]

In [72]:
for query in queries:
    result = app.invoke({"query": query})
    print(result)


Running Pharmaceutical RAG...
Error loading pharmaceutical data: [Errno 2] No such file or directory: 'pharmaceutical_database.json'
Routing Query...
Performing Web Search...
Routing Query...
Generating Answer...
{'query': 'What is the composition and primary use of Paracetamol?', 'chat_history': [{'role': 'user', 'content': 'What is the composition and primary use of Paracetamol?'}, {'role': 'ai', 'content': 'Based on the provided context, I can provide a detailed and accurate response regarding the composition and primary use of Paracetamol.\n\n**Composition:**\nParacetamol, also known as acetaminophen, is a non-opioid analgesic and antipyretic agent. Its chemical name is N-acetyl-para-aminophenol (APAP). It is a white, crystalline powder with a melting point of 154-156°C.\n\n**Primary Use:**\nThe primary use of Paracetamol is for the relief of mild to moderate pain, such as headache, muscle aches, minor arthritis pain, toothache, and pain caused by cold, flu, sprains, and dysmenorrh